## 什么是多任务？

简单地说，就是操作系统可以同时运行多个任务。打个比方，你一边在用浏览器上网，一边在听MP3，一边在用Word赶作业，这就是多任务，至少同时有3个任务正在运行。还有很多任务悄悄地在后台同时运行着，只是桌面上没有显示而已。

## 单核CPU如何执行多任务？

现在，多核CPU已经非常普及了，但是，即使过去的单核CPU，也可以执行多任务。由于CPU执行代码都是顺序执行的，那么，单核CPU是怎么执行多任务的呢？

答案就是操作系统轮流让各个任务交替执行，任务1执行0.01秒，切换到任务2，任务2执行0.01秒，再切换到任务3，执行0.01秒……这样反复执行下去。表面上看，每个任务都是交替执行的，但是，由于CPU的执行速度实在是太快了，我们感觉就像所有任务都在同时执行一样。

真正的并行执行多任务只能在多核CPU上实现，但是，由于任务数量远远多于CPU的核心数量，所以，操作系统也会自动把很多任务轮流调度到每个核心上执行。

## 进程和线程是什么？

对于操作系统来说，一个任务就是一个进程（Process），比如打开一个浏览器就是启动一个浏览器进程，打开一个记事本就启动了一个记事本进程，打开两个记事本就启动了两个记事本进程，打开一个Word就启动了一个Word进程。

有些进程还不止同时干一件事，比如Word，它可以同时进行打字、拼写检查、打印等事情。在一个进程内部，要同时干多件事，就需要同时运行多个“子任务”，我们把进程内的这些“子任务”称为线程（Thread）。


## 多线程的执行方式和多进程是一样的

由于每个进程至少要干一件事，所以，一个进程至少有一个线程。当然，像Word这种复杂的进程可以有多个线程，多个线程可以同时执行，多线程的执行方式和多进程是一样的，也是由操作系统在多个线程之间快速切换，让每个线程都短暂地交替运行，看起来就像同时执行一样。当然，真正地同时执行多线程需要多核CPU才可能实现。

## 同时执行多个任务怎么办？

有两种解决方案：

一种是启动多个进程，每个进程虽然只有一个线程，但多个进程可以一块执行多个任务。

还有一种方法是启动一个进程，在一个进程内启动多个线程，这样，多个线程也可以一块执行多个任务。

当然还有第三种方法，就是启动多个进程，每个进程再启动多个线程，这样同时执行的任务就更多了，当然这种模型更复杂，实际很少采用。

总结一下就是，多任务的实现有3种方式：

多进程模式；
多线程模式；
多进程+多线程模式。

同时执行多个任务通常各个任务之间并不是没有关联的，而是需要相互通信和协调，有时，任务1必须暂停等待任务2完成后才能继续执行，有时，任务3和任务4又不能同时执行，所以，多进程和多线程的程序的复杂度要远远高于我们前面写的单进程单线程的程序。

因为复杂度高，调试困难，所以，不是迫不得已，我们也不想编写多任务。但是，有很多时候，没有多任务还真不行。想想在电脑上看电影，就必须由一个线程播放视频，另一个线程播放音频，否则，单线程实现的话就只能先把视频播放完再播放音频，或者先把音频播放完再播放视频，这显然是不行的。

Python既支持多进程，又支持多线程

## 小结

线程是最小的执行单元，而进程由至少一个线程组成。如何调度进程和线程，完全由操作系统决定，程序自己不能决定什么时候执行，执行多长时间。

多进程和多线程的程序涉及到同步、数据共享的问题，编写起来更复杂。


## 多进程

Unix/Linux操作系统提供了一个fork()系统调用，它非常特殊。普通的函数调用，调用一次，返回一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程），然后，分别在父进程和子进程内返回。

子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。

Python的os模块封装了常见的系统调用，其中就包括fork，可以在Python程序中轻松创建子进程：



In [1]:
import os
os.fork()
print('I am learning multiprocesses')

# fork函数一旦运行就会生出一条新的进程，2个进程一起执行导致输出了2行。

I am learning multiprocesses
I am learning multiprocesses


## fork进程创建子进程后的程序流程

使用fork创建子进程后，子进程会复制父进程的数据信息，而后程序就分两个进程继续运行后面的程序，这也是fork（分叉）名字的含义了。在子进程内，这个方法会返回0；在父进程内，这个方法会返回子进程的编号PID。可以使用PID来区分两个进程

windows系统下没有fork()调用,so,只能在posix的系统下执行(Unix,Linux,Mac,BSD...)

os.getpid()是获取的是当前进程的进程号

os.getppid()是获取当前进程的父进程的进程号

In [2]:
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork() # 调用os.fork()函数
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
    # 在子进程内，os.fork()返回0
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))
    # 在父进程内，os.fork()会返回子进程的编号 ID    
    

Process (4037) start...
I (4037) just created a child process (4108).
I am child process (4108) and my parent is 4037.


## multiprocessing 多进程

### Process(\[group [, target [, name [, args [, kwargs]]]]])

target：如果传递了函数的引用，可以认为这个子进程就执行这里的代码

args：给target指定的函数传递的参数，以元组的方式传递

kwargs：给target指定的函数传递命名参数

name：给进程设定一个名字，可以不设定

group：指定进程组，大多数情况下用不到


### Process创建的实例对象的常用方法

start()：启动子进程实例（创建子进程）

is_alive()：判断进程子进程是否还在活着

join([timeout])：是否等待子进程执行结束，或等待多少秒

join方法的作用是阻塞，等待子线程结束，join方法有一个参数是timeout，即如果主线程等待timeout，子线程还没有结束，则主线程强制结束子线程。

terminate()：不管任务是否完成，立即终止子进程


### Process创建的实例对象的常用属性：

name：当前进程的别名，默认为Process-N，N为从1开始递增的整数

pid：当前进程的pid（进程号）

In [4]:
from multiprocessing import Process
import os

#子进程要执行的代码
def run_proc(name):
    print('Run child process {} ({})'.format(name,os.getpid()))
    
print('Parent process {}'.format(os.getpid()))
p = Process(target=run_proc,args=('text',))
print('Child Process will start.')
p.start  #启动进程
p.join() #等待子进程结束后再继续往下运行，通常用于进程间的同步。
print('Child process end.')
    

Parent process 4037
Child Process will start.
Run child process text (4174)
Child process end.


创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，这样创建进程比fork()还要简单。

join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。

## Pool 进程池

如果要启动大量的子进程，可以用进程池的方式批量创建子进程

In [17]:
from multiprocessing import Pool
import os,time,random

def long_time_task(name):
    print('Run task {} ({})'.format(name,os.getpid()))
    start = time.time()
    time.sleep(random.random()*3) #随机暂停
    end = time.time()
    print('Task {} runs {} seconds'.format(name,(end-start)))

print('Parent procee {}'.format(os.getpid()))
p = Pool() #创建进程池
for i in range(5):
    p.apply_async(long_time_task,args=(i,))# 将要执行的函数加入进程池
print('Waiting for all subprocess done...')
p.close()
p.join() #等待所有子进程执行完毕,之前必须先调用close(),然后不能添加新的Process.
print('All subprocesses done.')

Parent procee 4037
Run task 1 (4221)
Run task 0 (4220)
Run task 3 (4223)
Run task 2 (4222)
Waiting for all subprocess done...
Task 0 runs 0.12268900871276855 seconds
Run task 4 (4220)
Task 2 runs 1.173778772354126 seconds
Task 3 runs 1.4420840740203857 seconds
Task 1 runs 2.0074009895324707 seconds
Task 4 runs 2.5265331268310547 seconds
All subprocesses done.


In [3]:
import time

def calc_square(numbers):
    print('calaculate square numbers')
    for n in numbers:
        time.sleep(0.02)
        print('square:{}'.format(n*n))
        
def calc_cube(numbers):
    print('calaculate cube numbers')
    for n in numbers:
        time.sleep(0.02)
        print('cube:{}'.format(n*n*n))

arr = [2,3,8,9]

t = time.time()
calc_square(arr)
calc_cube(arr)
print('done in:{}'.format(time.time()-t))
print('Hah... I am done with all my work now!')

calaculate square numbers
square:4
square:9
square:64
square:81
calaculate cube numbers
cube:8
cube:27
cube:512
cube:729
done in:0.16292309761047363
Hah... I am done with all my work now!


In [27]:
import time
import threading

result = []

def calc_square(numbers):
    global result
    print('calaculate square numbers')
    for n in numbers:
        time.sleep(0.02)
        print('square:{}'.format(n*n))
        result.append(n*n)
    print('inside thread result'+str(result))

        
def calc_cube(numbers):
    print('calaculate cube numbers')
    for n in numbers:
        time.sleep(0.02)
        print('cube:{}'.format(n*n*n))

arr = [2,3,8,9]

t = time.time()
t1 = threading.Thread(target=calc_square,args=(arr,))
#t2 = threading.Thread(target=calc_cube,args=(arr,))
t1.start()
#t2.start()
t1.join()
#t2.join()

print('outside thread result:'+ str(result))
print('done in:{}'.format(time.time()-t))
print('Hah... I am done with all my work now!')

calaculate square numbers
calaculate cube numbers
square:4
cube:8
square:9
cube:27
square:64
cube:512
square:81
inside thread result[4, 9, 64, 81]
cube:729
outside thread result:[4, 9, 64, 81]
done in:0.10058450698852539
Hah... I am done with all my work now!
